as a continuation of [feature_extraction_20200523](./feature_extraction_20200523.ipynb), this notebook examines in more detail certain models, regarding convergence of recurrent computations.

as a first step, it will focus on

1. models with a high number of recurrent cycles (5,6,7)
2. models with relatively smaller cycle-to-cycle differences. "cycle-to-cycle differences" can at least be defined in two ways. I hope two definitions lead to consistent results.
    1. taking response map as a whole (N x C x D x D) and check its relative change across iterations.
    2. taking response map of each of N images, and check **average** relative change across images.

In [1]:
from os.path import dirname, relpath, realpath, join, exists

import h5py

from numpy.linalg import norm
import numpy as np

from thesis_v2 import dir_dict
from thesis_v2.submission import utils
from thesis_v2.configs.model.maskcnn_polished_with_rcnn_k_bl import (
    explored_models_20200523_8k_feature_extraction_generator,
    script_keygen,
    gen_feature_extraction_global_vars
)
from json import dump, load
from os import remove

In [2]:
def generate_diff_vec(*, tensor_list, acc_mode, max_len):
    assert len(tensor_list) > 1
    
    if acc_mode=='instant':
        pass
    elif acc_mode == 'cummean':
        tensor_list_new = []
        for i in range(len(tensor_list)):
            tensor_list_new.append(np.mean(np.asarray(tensor_list[:i+1]), axis=0))
        tensor_list = tensor_list_new
    else:
        raise ValueError
    
    tensor_shape = tensor_list[0].shape
    # get norm diff
    diff_vec = np.full((max_len,), fill_value=np.nan, dtype=np.float64)
    
    for i in range(len(tensor_list)-1):
        vec_prev = tensor_list[i]
        vec_now = tensor_list[i+1]
        assert vec_prev.shape == vec_now.shape
        assert vec_prev.ndim == 3
        diff_this = norm(vec_now.ravel()-vec_prev.ravel())/norm(vec_prev.ravel())
        diff_vec[i] = diff_this
    return diff_vec

def generate_diff_vec_per_image(*, tensor_list, acc_mode, max_len):
    num_img = tensor_list[0].shape[0]
    assert num_img == 1600
    diff_vec_list = []
    for idx_img in range(num_img):
        tensor_list_this_img = [x[idx_img] for x in tensor_list]
        diff_vec_list.append(generate_diff_vec(
            tensor_list = tensor_list_this_img,
            acc_mode=acc_mode,
            max_len = max_len,
        ))
    diff_vec_list = np.asarray(diff_vec_list)
    assert diff_vec_list.shape == (num_img, max_len)
    diff_vec_mean = diff_vec_list.mean(axis=0)
    diff_vec_std = diff_vec_list.std(axis=0)
    return {
        'mean': diff_vec_mean.tolist(),
        'std': diff_vec_std.tolist(),
    }
    

def load_all():
    
    batch_key='yuanyuan_8k_a/20200523'
    global_vars_for_feature_extraction = gen_feature_extraction_global_vars(key=batch_key)
    
    for idx, param_dict in enumerate(explored_models_20200523_8k_feature_extraction_generator()):
        
        cls = param_dict['rcnn_bl_cls']
        # only examine 5, 6, 7
        if cls <= 4:
            continue
        
        key_this = script_keygen(**param_dict)        
        h5file = join(global_vars_for_feature_extraction['feature_file_dir'], key_this+'.hdf5')
        
        auxfile = join(global_vars_for_feature_extraction['feature_file_dir'], key_this+'.json')
        assert exists(auxfile)
        auxfile2 = join(global_vars_for_feature_extraction['feature_file_dir'], key_this +'.per_image.json')
        assert auxfile2 != auxfile
        if not exists(auxfile2):
            with h5py.File(h5file, 'r') as f:
                g = f['test']
                resp_map = [g[f'0.{x}'][()] for x in range(cls)]
                diff_vec_map_instant_dict = generate_diff_vec_per_image(
                    tensor_list = resp_map,
                    acc_mode='instant',
                    max_len = 6,
                )

                diff_vec_map_cummean_dict = generate_diff_vec_per_image(
                    tensor_list = resp_map,
                    acc_mode='cummean',
                    max_len = 6,
                )
            # write aux file.
            try:
                with open(auxfile2, 'wt', encoding='utf-8') as f_aux:
                    dump({
                        'test': {
                            'diff_bl_stack_instant': diff_vec_map_instant_dict,
                            'diff_bl_stack_cummean': diff_vec_map_cummean_dict,
                        }
                    },
                        f_aux,
                    )
            except Exception:
                if exists(auxfile2):
                    remove(auxfile2)
                
        with open(auxfile, 'rt', encoding='utf-8') as f_aux:
            aux_obj = load(f_aux)
            
        with open(auxfile2, 'rt', encoding='utf-8') as f_aux2:
            aux_obj2 = load(f_aux2)
        
        if idx % 10 == 0:
            print(idx)
            print(aux_obj)
            print(aux_obj2)
load_all()

1160
{'test': {'diff_final_act': [0.03723845258355141, 0.024219166487455368, 0.01984962821006775, 0.018368912860751152, 0.01686611957848072, nan], 'diff_bl_stack_instant': [0.31448879837989807, 0.22004656493663788, 0.19643515348434448, 0.17911124229431152, 0.16675014793872833, nan], 'diff_bl_stack_cummean': [0.15724439918994904, 0.09497424960136414, 0.07939436286687851, 0.06908543407917023, 0.06217724457383156, nan]}}
{'test': {'diff_bl_stack_instant': {'mean': [0.3141497570835054, 0.22162918016314506, 0.19751474052667617, 0.18032879602164031, 0.16778818488121033, nan], 'std': [0.021961500608383387, 0.023060350176144224, 0.017436865969331587, 0.0201869395323359, 0.014516447683032044, nan]}, 'diff_bl_stack_cummean': {'mean': [0.15707487864419817, 0.09536005500238388, 0.07944633719045668, 0.06901542078237981, 0.0620853238995187, nan], 'std': [0.010980750384111084, 0.006853865406249004, 0.005482314651504078, 0.005253867744260768, 0.00426690878459761, nan]}}}
1170
{'test': {'diff_final_act